In [38]:
import numpy as np
import torch
import os
import trimesh

MOTION_PATH = 'data/neuraldome/sequences_canonical'
OBJECT_PATH = 'data/neuraldome/objects'
OBJECT_BPS_PATH = 'data/neuraldome/objects_bps'
name = 'subject01_baseball_0'
obj_name = 'baseball'

    

In [30]:
# Load human raw data
with np.load(os.path.join(MOTION_PATH, name, 'human.npz'), allow_pickle=True) as f:
    poses = f['poses']                # smplh(or smplx) poses
    betas = f['betas']                # body shape
    trans = f['trans']                # global body position
    gender = str(f['gender'])         # gender

# Load object raw data
with np.load(os.path.join(MOTION_PATH, name, 'object.npz'), allow_pickle=True) as f:
    obj_angles = f['angles']          # rotation for rigid object, axis angle
    obj_trans = f['trans']            # translation for rigid object
    obj_name = str(f['name'])         # object name


print("=" * 50)
print("Variable Information".center(50))
print("=" * 50)
print(f"Poses shape: {poses.shape}")
print(f"Betas shape: {betas.shape}")
print(f"Trans shape: {trans.shape}")
print(f"Gender: {gender}")
print(f"Object angles shape: {obj_angles.shape}")
print(f"Object translation shape: {obj_trans.shape}")
print(f"Object name: {obj_name}")
print("=" * 50)


               Variable Information               
Poses shape: (190, 156)
Betas shape: (16,)
Trans shape: (190, 3)
Gender: neutral
Object angles shape: (190, 3)
Object translation shape: (190, 3)
Object name: baseball


In [12]:
# Load the text annotations
with open(os.path.join(MOTION_PATH, name, 'text.txt'), 'r') as f:
    texts = f.readlines()             # Read all the texts for this sequence
texts

['A person picks up a baseball with right hand, holds it with left hand, and touches their head with their right hand.#A/DET person/NOUN pick/VERB up/ADP a/DET baseball/NOUN with/ADP right/ADJ hand/NOUN hold/VERB it/PRON with/ADP left/ADJ hand/NOUN and/CCONJ touch/VERB their/PRON head/NOUN with/ADP their/PRON right/ADJ hand/NOUN#0.0#0.0\n',
 'An individual grasps a baseball with their right hand, secures it with their left, and then touches their head with the right hand.#An/DET individual/NOUN grasp/VERB a/DET baseball/NOUN with/ADP their/PRON right/ADJ hand/NOUN secure/VERB it/PRON with/ADP their/PRON left/ADJ and/CCONJ then/ADV touch/VERB their/PRON head/NOUN with/ADP the/DET right/ADJ hand/NOUN#0.0#0.0\n',
 'A person picks up a baseball with their right hand, holds it with their left, and touches their head with the right.#A/DET person/NOUN pick/VERB up/ADP a/DET baseball/NOUN with/ADP their/PRON right/ADJ hand/NOUN hold/VERB it/PRON with/ADP their/PRON left/ADJ and/CCONJ touch/VER

In [19]:
# Load the action annotations
with open(os.path.join(MOTION_PATH, name, 'action.txt'), 'r') as f:
    action = f.readlines()             # Read the action for this sequence
action

['A person move the chair.#A/DET person/NOUN move/VERB the/DET chair/NOUN#0.0#0.0']

In [33]:
# Load the human joints and markers
human_markers = np.load(os.path.join(MOTION_PATH, name, 'markers.npy'))
human_joints = np.load(os.path.join(MOTION_PATH, name, 'joints.npy'))
print("=" * 50)
print("Variable Information".center(50))
print("=" * 50)
print("Markers shape:",human_markers.shape)
print("Joints shape:",human_joints.shape)
print("=" * 50)

               Variable Information               
Markers shape: (190, 77, 3)
Joints shape: (190, 52, 3)


In [44]:
object_sample_verts = np.load(os.path.join(OBJECT_PATH, obj_name, 'sample_points.npy'))
MESH = trimesh.load(os.path.join(OBJECT_PATH, obj_name, f'{obj_name}.obj'))
object_verts = MESH.vertices
object_faces = MESH.faces
object_bps = np.load(os.path.join(OBJECT_BPS_PATH, obj_name, f'{obj_name}_1024.npy'))


print("=" * 50)
print("Variable Information".center(50))
print("=" * 50)
print('Object sampled verts:',object_sample_verts.shape )
print('Object verts:', object_verts.shape )
print('Object faces:', object_faces.shape )
print('Object BPS:', object_bps.shape )
print("=" * 50)

               Variable Information               
Object sampled verts: (340, 3)
Object verts: (12642, 3)
Object faces: (19954, 3)
Object BPS: (1, 1024, 3)


In [32]:
# Load the representation
motion = np.load(os.path.join(MOTION_PATH, name, 'motion.npy'))
print("=" * 50)
print("Variable Information".center(50))
print("=" * 50)
print("Representation shape:", motion.shape)
print("=" * 50)

               Variable Information               
Representation shape: (190, 962)




The `motion.npy` file contains motion sequence data in the shape of `(N, D)`, where:

- **`N`**: Represents the number of frames in the sequence.
- **`D`**: Denotes the feature dimension, which is **962** in total.   

This dimension is comprised of two parts:

- Human motion (476 features): 
  - Marker positions (231 features): 73 markers, each with 3 spatial coordinates.
  - Marker velocities (231 features): The velocity of each of the 73 markers.
  - Ground contact (14 features): Information related to the subject's contact with the ground.
- Object motion (486 features): 
  - Translation and 6D rotation (9 features): Representing the object's position and orientation.
  - Velocity (9 features): The translational and rotational velocity of the object.
  - Relative representation (468 features): 78 relative representations(77 + 1 ground marker) , each with 6 components.

In [31]:
# Reshape and extract variables from the 'motion' array

# Markers positions:
# Reshaped into (-1, 77, 3) meaning there are 77 markers with 3 coordinates each.
markers = motion[:, :231].reshape(-1, 77, 3)

# Markers velocities:
# Reshaped into (-1, 77, 3) corresponding to 77 markers each having a 3D velocity vector.
markers_velocity = motion[:, 231:462].reshape(-1, 77, 3)

# Marker foot contact:
# Contains 14 values indicating foot contact statuses for markers.
marker_foot_contact = motion[:, 462:462+14]

# Object rotation (6D):
# Represents object rotation using a 6D continuous representation.
object_rotation_6d = motion[:, 476:476+6]

# Object translation:
# Contains the object's 3D translation values.
object_trans = motion[:, 482:485]

# Object angular velocity (6D):
# Represents the object's angular velocity in a 6-dimensional format.
object_ang_vel_6d = motion[:, 485:491]

# Object velocity:
# Contains the object's 3D linear velocity.
object_vel = motion[:, 491:494]

# Vectors from the object to markers:
# Reshaped into (-1, 77, 3) yielding a 3D vector for each of the 77 markers.
object2marker = motion[:, 494:494+77*3].reshape(-1, 77, 3)

# Vector from the object to the ground:
# Reshaped into (-1, 1, 3) representing the 3D vector for ground contact.
object2ground = motion[:, 725:725+3].reshape(-1, 1, 3)

# Canonical Object vertices corresponding to each marker:
# Reshaped into (-1, 77, 3) where each marker corresponds to one vertex (with 3 coordinates).
marker_correspond_canonical_object_verts = motion[:, 728:728+77*3].reshape(-1, 77, 3)

# Canonical Object vertex corresponding to the ground contact,:
# Reshaped into (-1, 1, 3) for the ground-related object vertex.
ground_correspond_canonical_object_verts = motion[:, 959:959+3].reshape(-1, 1, 3)

print("=" * 50)
print("Variable Information".center(50))
print("=" * 50)
print("Markers shape:", markers.shape)
print("Markers velocity shape:", markers_velocity.shape)
print("Marker foot contact shape:", marker_foot_contact.shape)
print("Object rotation (6D) shape:", object_rotation_6d.shape)
print("Object translation shape:", object_trans.shape)
print("Object angular velocity (6D) shape:", object_ang_vel_6d.shape)
print("Object velocity shape:", object_vel.shape)
print("Object-to-marker vectors shape:", object2marker.shape)
print("Object-to-ground vector shape:", object2ground.shape)
print("Marker-corresponding canonical object vertices shape:", marker_correspond_canonical_object_verts.shape)
print("Ground-corresponding canonical object vertex shape:", ground_correspond_canonical_object_verts.shape)
print("=" * 50)

               Variable Information               
Markers shape: (190, 77, 3)
Markers velocity shape: (190, 77, 3)
Marker foot contact shape: (190, 14)
Object rotation (6D) shape: (190, 6)
Object translation shape: (190, 3)
Object angular velocity (6D) shape: (190, 6)
Object velocity shape: (190, 3)
Object-to-marker vectors shape: (190, 77, 3)
Object-to-ground vector shape: (190, 1, 3)
Marker-corresponding canonical object vertices shape: (190, 77, 3)
Ground-corresponding canonical object vertex shape: (190, 1, 3)
